In [ ]:
import weave
import pandas as pd
from ydata_profiling import ProfileReport

from ydnpd import load_dataset
from ydnpd.agent.specifications import SPECIFICATION_V0
from ydnpd.agent.utils import produce_mixture_dataset

RANDOM_STATE = 42

weave.init("data_gen_agent")


In [ ]:
PRIVATE_DATASET = "acs/national"
num_samples = len(load_dataset(PRIVATE_DATASET)[0])

LLM_KWARGS = {"llm_name": "gpt-4o-mini"}

# LLM_KWARGS = {"llm_name": "o1-preview", 
#               "llm_temperature": 1,
#               "llm_max_tokens": None}

In [ ]:
mixture_df, details = produce_mixture_dataset(PRIVATE_DATASET, SPECIFICATION_V0, num_samples, 2,
                                              **LLM_KWARGS)

In [ ]:
# mixture_df.to_csv("ABC.csv")

In [ ]:
(ProfileReport(load_dataset(PRIVATE_DATASET)[0], title="Private")
 .compare(
     ProfileReport(mixture_df, title="Agent")
 )
)